In [4]:
pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 75.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.6 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=6696606c09e7e2f2462a3382ac3b5100c37262c8828d076d4359b38c3d2dee1f
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [5]:
import pandas as pd
import time
from tqdm import tqdm
import seaborn as sns
import numpy as np
from textblob import TextBlob
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer

In [9]:
from google.colab import files
uploaded = files.upload()

Saving wiki_movie_plots_deduped.csv to wiki_movie_plots_deduped.csv


In [6]:
!pip install faiss-cpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 32.9 MB/s eta 0:00:00


In [7]:
import faiss

In [22]:
data = pd.read_csv("wiki_movie_plots_deduped.csv")

# Clean data
df = data[['Title','Plot']]
df.dropna(inplace=True).drop_duplicates(subset=['Plot'],inplace=True)
df = df.head(100)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34886 entries, 0 to 34885
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Release Year      34886 non-null  int64 
 1   Title             34886 non-null  object
 2   Origin/Ethnicity  34886 non-null  object
 3   Director          34886 non-null  object
 4   Cast              33464 non-null  object
 5   Genre             34886 non-null  object
 6   Wiki Page         34886 non-null  object
 7   Plot              34886 non-null  object
dtypes: int64(1), object(7)
memory usage: 2.1+ MB


<ipython-input-22-98ae7c4e0d0d>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
<ipython-input-22-98ae7c4e0d0d>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(subset=['Plot'],inplace=True)


In [23]:
len(df)

33869

In [11]:
model = SentenceTransformer('msmarco-distilbert-base-dot-prod-v3')

In [12]:
encoded_data = model.encode(df.Plot.tolist())
encoded_data = np.asarray(encoded_data.astype('float32'))
index = faiss.IndexIDMap(faiss.IndexFlatIP(768))
index.add_with_ids(encoded_data, np.array(range(0, len(df))))
faiss.write_index(index, 'movie_plot.index')

In [29]:
def search(query, top_k, index, model):
    t=time.time()
    query_vector = model.encode([query])
    top_k = index.search(query_vector, top_k)
    print('Time: {}'.format(time.time()-t))
    top_k_ids = top_k[1].tolist()[0]
    top_k_ids = list(np.unique(top_k_ids))
    results =  [fetch_movie_info(idx) for idx in top_k_ids]
    return results

In [30]:
query="what are the top romantic movies"
results=search(query, top_k=5, index=index, model=model)
results

Time: 0.0533907413482666


[{'Title': 'Love by the Light of the Moon'},
 {'Title': 'For Her Sake'},
 {'Title': 'The Land Beyond the Sunset'},
 {'Title': 'Home, Sweet Home'},
 {'Title': 'The Property Man'}]

In [13]:
model_2 = SentenceTransformer("")

In [14]:
encoded_data = model_2.encode(df.Plot.tolist())
encoded_data = np.asarray(encoded_data.astype('float32'))
index = faiss.IndexIDMap(faiss.IndexFlatIP(768))
index.add_with_ids(encoded_data, np.array(range(0, len(df))))

In [17]:
query="what are the top romantic movies"
results=search(query, top_k=5, index=index, model=model_2)
results

>>>> Results in Total Time: 0.08126997947692871


	 {'Title': 'Love by the Light of the Moon'}
	 {'Title': 'For Her Sake'}
	 {'Title': 'The Land Beyond the Sunset'}
	 {'Title': 'Home, Sweet Home'}
	 {'Title': 'The Property Man'}


In [20]:
model_3 = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")

In [21]:
encoded_data = model_2.encode(df.Plot.tolist())
encoded_data = np.asarray(encoded_data.astype('float32'))
index = faiss.IndexIDMap(faiss.IndexFlatIP(768))

In [31]:
query="what are the top romantic movies"
results=search(query, top_k=5, index=index, model=model_3)
results

Time: 0.11793851852416992


[{'Title': 'The Suburbanite'},
 {'Title': 'A Calamitous Elopement'},
 {'Title': 'At the Altar'},
 {'Title': 'For Her Sake'},
 {'Title': 'The Musketeers of Pig Alley'}]